# Rock-Paper-Scissor Classifier

The objective of this study is, by using  a two hidden layer neural network, to classify images of hands in 3 classes : rock, paper and scissors.

Authors : Hamza Boukraichi and Loic Cordone

In [1]:
import numpy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim

from PIL import Image
from torch.utils.data.dataset import Dataset
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torchvision import transforms, datasets
import matplotlib.pyplot as plt

# Create the image folder and the loaders



In [2]:
transf = transforms.Compose([transforms.Grayscale(),
                             transforms.RandomRotation(180),
                             transforms.RandomHorizontalFlip(),
                             transforms.ToTensor()])
dataset = datasets.ImageFolder('PPC2/.',transform = transf)

#Splitting DataSet into Train  and Test
n = len(dataset)
ntrain = int(0.8 *n)
ntest = n - ntrain
train_data, test_data = torch.utils.data.random_split(dataset, [ntrain, ntest])
nbatch = 10


In [3]:
dataload_train = dict(shuffle=True, batch_size=nbatch,num_workers=0, pin_memory=True)
train_load = dataloader.DataLoader(train_data, **dataload_train)

dataload_test = dict(shuffle=True, batch_size=ntest,num_workers=0, pin_memory=True)
test_load = dataloader.DataLoader(test_data, **dataload_test)

# Build the model



In [4]:
# Two hidden Layers NN
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc0 = nn.Linear(32*32, 16*16)
        self.fc1 = nn.Linear(16*16, 8*8)
        self.fcout = nn.Linear(8*8,3)
    def forward(self, x):
        out = self.fc0(x.view(-1,32*32))
        out = F.relu(self.fc1(out))
        out = F.relu(self.fcout(out))
        return F.log_softmax(out, dim=0)    
    
    
model = Model()
optimizer = optim.Adagrad(model.parameters())

EPOCHS = 20
losses = []


#  Train & Test


In [5]:
# Train the model
total_step = len(train_load)
for epoch in range(EPOCHS):
    losses = []
    for i, (images, labels) in enumerate(train_load):

        # Forward pass
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        losses.append(loss.item())
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
   

    # Test the model
    model.eval()  # eval mode
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_load:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Epoch [{}/{}], \t Loss: {:.4f}, \t Accuracy On Test:{:.4f}%' 
                .format(epoch+1, EPOCHS, np.mean(losses),100 * correct / total))

Epoch [1/20], 	 Loss: 1.1310, 	 Accuracy On Test:29.2683%
Epoch [2/20], 	 Loss: 1.0986, 	 Accuracy On Test:29.2683%
Epoch [3/20], 	 Loss: 1.0986, 	 Accuracy On Test:29.2683%
Epoch [4/20], 	 Loss: 1.0986, 	 Accuracy On Test:29.2683%
Epoch [5/20], 	 Loss: 1.0986, 	 Accuracy On Test:29.2683%
Epoch [6/20], 	 Loss: 1.0986, 	 Accuracy On Test:29.2683%
Epoch [7/20], 	 Loss: 1.0986, 	 Accuracy On Test:29.2683%
Epoch [8/20], 	 Loss: 1.0986, 	 Accuracy On Test:29.2683%
Epoch [9/20], 	 Loss: 1.0986, 	 Accuracy On Test:29.2683%
Epoch [10/20], 	 Loss: 1.0986, 	 Accuracy On Test:29.2683%
Epoch [11/20], 	 Loss: 1.0986, 	 Accuracy On Test:29.2683%
Epoch [12/20], 	 Loss: 1.0986, 	 Accuracy On Test:29.2683%
Epoch [13/20], 	 Loss: 1.0986, 	 Accuracy On Test:29.2683%
Epoch [14/20], 	 Loss: 1.0986, 	 Accuracy On Test:29.2683%
Epoch [15/20], 	 Loss: 1.0986, 	 Accuracy On Test:29.2683%
Epoch [16/20], 	 Loss: 1.0986, 	 Accuracy On Test:29.2683%
Epoch [17/20], 	 Loss: 1.0986, 	 Accuracy On Test:29.2683%
Epoch 